<a href="https://www.kaggle.com/code/udaykirankavaturu/playground-series-season-3-episode-19?scriptVersionId=136815865" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e19/sample_submission.csv
/kaggle/input/playground-series-s3e19/train.csv
/kaggle/input/playground-series-s3e19/test.csv


In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s3e19/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv')

### Non graphical EDA

In [10]:
train_data.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


train data has day-wise sales of a product in a specific store in a specific country. this is a time-series data. 

In [11]:
test_data.head()

,id,date,country,store,product
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better


test data has all columns as the train_data except for num_sold which needs to be predicted. 

In [4]:
train_data.shape

(136950, 6)

train data has 136950 rows and 6 columns

In [5]:
test_data.shape

(27375, 5)

test data has 27375 rows and 5 columns

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        136950 non-null  int64 
 1   date      136950 non-null  object
 2   country   136950 non-null  object
 3   store     136950 non-null  object
 4   product   136950 non-null  object
 5   num_sold  136950 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.3+ MB


train data has no null values. 

In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375 entries, 0 to 27374
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       27375 non-null  int64 
 1   date     27375 non-null  object
 2   country  27375 non-null  object
 3   store    27375 non-null  object
 4   product  27375 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


test data also has no null values

In [8]:
train_data.describe()

,id,num_sold
count,136950.00000,136950.000000
mean,68474.50000,165.522636
std,39534.20402,183.691575
min,0.00000,2.000000
25%,34237.25000,46.000000
50%,68474.50000,98.000000
75%,102711.75000,184.000000
max,136949.00000,1380.000000


there is only one numeric column num_sold. num_sold ranges from 2-1380 with an average of 165. 

In [9]:
test_data.describe()

,id
count,27375.000000
mean,150637.000000
std,7902.626146
min,136950.000000
25%,143793.500000
50%,150637.000000
75%,157480.500000
max,164324.000000


only id is available here. num_sold needs to be predicted. 

In [20]:
print(train_data.date.min())
print(train_data.date.max())

2017-01-01
2021-12-31


date range spans from 1st January 2017 to 31st December 2021. it's a 5 year data set. 

In [21]:
print(test_data.date.min())
print(test_data.date.max())

2022-01-01
2022-12-31


date range spans from 1st January 2022 to 31st December 2022. it's a 1 year data set which needs to be predicted. 

In [22]:
train_data.country.value_counts()

Argentina    27390
Canada       27390
Estonia      27390
Japan        27390
Spain        27390
Name: country, dtype: int64

there are 5 countries in the train data. all countries have equal number of entries. 

In [23]:
test_data.country.value_counts()

Argentina    5475
Canada       5475
Estonia      5475
Japan        5475
Spain        5475
Name: country, dtype: int64

there are 5 countries in the test data as well. the spread is also equal. 

In [24]:
train_data.store.value_counts()

Kaggle Learn    45650
Kaggle Store    45650
Kagglazon       45650
Name: store, dtype: int64

there are 3 stores in the train data. all stores have equal number of entries.

In [25]:
test_data.store.value_counts()

Kaggle Learn    9125
Kaggle Store    9125
Kagglazon       9125
Name: store, dtype: int64

there are 3 stores in the test data. the spread is equal.

In [28]:
train_data['product'].value_counts()

Using LLMs to Improve Your Coding                 27390
Using LLMs to Train More LLMs                     27390
Using LLMs to Win Friends and Influence People    27390
Using LLMs to Win More Kaggle Competitions        27390
Using LLMs to Write Better                        27390
Name: product, dtype: int64

there are 5 types of products in the train data. all products have equal number of entries. 

In [29]:
test_data['product'].value_counts()

Using LLMs to Improve Your Coding                 5475
Using LLMs to Train More LLMs                     5475
Using LLMs to Win Friends and Influence People    5475
Using LLMs to Win More Kaggle Competitions        5475
Using LLMs to Write Better                        5475
Name: product, dtype: int64

there are 5 types of products in the test data as well. all products have equal number of entries. 

In [31]:
train_data.groupby(by='date').count()

,id,country,store,product,num_sold
date,,,,,
2017-01-01,75,75,75,75,75
2017-01-02,75,75,75,75,75
2017-01-03,75,75,75,75,75
2017-01-04,75,75,75,75,75
2017-01-05,75,75,75,75,75
...,...,...,...,...,...
2021-12-27,75,75,75,75,75
2021-12-28,75,75,75,75,75
2021-12-29,75,75,75,75,75


looks like there are 75 entries per each date in the train data. 

In [32]:
test_data.groupby(by='date').count()

,id,country,store,product
date,,,,
2022-01-01,75,75,75,75
2022-01-02,75,75,75,75
2022-01-03,75,75,75,75
2022-01-04,75,75,75,75
2022-01-05,75,75,75,75
...,...,...,...,...
2022-12-27,75,75,75,75
2022-12-28,75,75,75,75
2022-12-29,75,75,75,75


there are 75 entries per each date in the test data as well. 

In [36]:
train_data.groupby(by=['date','country']).count()

id  store  product  num_sold
date       country                                
2017-01-01 Argentina  15     15       15        15
           Canada     15     15       15        15
           Estonia    15     15       15        15
           Japan      15     15       15        15
           Spain      15     15       15        15
...                   ..    ...      ...       ...
2021-12-31 Argentina  15     15       15        15
           Canada     15     15       15        15
           Estonia    15     15       15        15
           Japan      15     15       15        15
           Spain      15     15       15        15

[9130 rows x 4 columns]

there are 15 entries per date per country in the train data.

In [37]:
test_data.groupby(by=['date','country']).count()

id  store  product
date       country                      
2022-01-01 Argentina  15     15       15
           Canada     15     15       15
           Estonia    15     15       15
           Japan      15     15       15
           Spain      15     15       15
...                   ..    ...      ...
2022-12-31 Argentina  15     15       15
           Canada     15     15       15
           Estonia    15     15       15
           Japan      15     15       15
           Spain      15     15       15

[1825 rows x 3 columns]

there are 15 entries per date per country in the test data as well. 

In [38]:
train_data.groupby(by=['country','product']).count()

id  date  store  \
country   product                                                             
Argentina Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Canada    Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Estonia   Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Japan     Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Spain     Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   

                                                          num_sold  
country   product                                                   
Argentina Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Canada    Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Estonia   Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Japan     Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Spain     Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478

there are 5478 entries per country per product in the train data.

In [39]:
test_data.groupby(by=['country','product']).count()

id  date  store
country   product                                                          
Argentina Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Canada    Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Estonia   Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Japan     Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Spain     Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095

there are 1095 entries per country per product in the test data.

In [40]:
train_data.groupby(by=['product','num_sold']).count()

id  date  country  store
product                           num_sold                          
Using LLMs to Improve Your Coding 12         1     1        1      1
                                  13         6     6        6      6
                                  14         9     9        9      9
                                  15        16    16       16     16
                                  16        25    25       25     25
...                                         ..   ...      ...    ...
Using LLMs to Write Better        936        1     1        1      1
                                  957        1     1        1      1
                                  982        1     1        1      1
                                  1005       1     1        1      1
                                  1009       1     1        1      1

[3680 rows x 4 columns]

there seems to be some sort of non distribution of sales among products. we'll get into the details later. 

In [42]:
train_data.groupby(by=['date','store']).count()

id  country  product  num_sold
date       store                                       
2017-01-01 Kagglazon     25       25       25        25
           Kaggle Learn  25       25       25        25
           Kaggle Store  25       25       25        25
2017-01-02 Kagglazon     25       25       25        25
           Kaggle Learn  25       25       25        25
...                      ..      ...      ...       ...
2021-12-30 Kaggle Learn  25       25       25        25
           Kaggle Store  25       25       25        25
2021-12-31 Kagglazon     25       25       25        25
           Kaggle Learn  25       25       25        25
           Kaggle Store  25       25       25        25

[5478 rows x 4 columns]

there are 25 entries per date per store in the train data. 

In [43]:
test_data.groupby(by=['date','store']).count()

id  country  product
date       store                             
2022-01-01 Kagglazon     25       25       25
           Kaggle Learn  25       25       25
           Kaggle Store  25       25       25
2022-01-02 Kagglazon     25       25       25
           Kaggle Learn  25       25       25
...                      ..      ...      ...
2022-12-30 Kaggle Learn  25       25       25
           Kaggle Store  25       25       25
2022-12-31 Kagglazon     25       25       25
           Kaggle Learn  25       25       25
           Kaggle Store  25       25       25

[1095 rows x 3 columns]

there are 25 entries per date per store in the test data as well. 

In [46]:
train_data.groupby(by=['date','product']).count()

id  country  store  \
date       product                                                              
2017-01-01 Using LLMs to Improve Your Coding               15       15     15   
           Using LLMs to Train More LLMs                   15       15     15   
           Using LLMs to Win Friends and Influence People  15       15     15   
           Using LLMs to Win More Kaggle Competitions      15       15     15   
           Using LLMs to Write Better                      15       15     15   
...                                                        ..      ...    ...   
2021-12-31 Using LLMs to Improve Your Coding               15       15     15   
           Using LLMs to Train More LLMs                   15       15     15   
           Using LLMs to Win Friends and Influence People  15       15     15   
           Using LLMs to Win More Kaggle Competitions      15       15     15   
           Using LLMs to Write Better                      15       15     15   

                                                           num_sold  
date       product                                                   
2017-01-01 Using LLMs to Improve Your Coding                     15  
           Using LLMs to Train More LLMs                         15  
           Using LLMs to Win Friends and Influence People        15  
           Using LLMs to Win More Kaggle Competitions            15  
           Using LLMs to Write Better                            15  
...                                                             ...  
2021-12-31 Using LLMs to Improve Your Coding                     15  
           Using LLMs to Train More LLMs                         15  
           Using LLMs to Win Friends and Influence People        15  
           Using LLMs to Win More Kaggle Competitions            15  
           Using LLMs to Write Better                            15  

[9130 rows x 4 columns]

there are 15 entries per date per product in the train data.

In [47]:
test_data.groupby(by=['date','product']).count()

id  country  store
date       product                                                           
2022-01-01 Using LLMs to Improve Your Coding               15       15     15
           Using LLMs to Train More LLMs                   15       15     15
           Using LLMs to Win Friends and Influence People  15       15     15
           Using LLMs to Win More Kaggle Competitions      15       15     15
           Using LLMs to Write Better                      15       15     15
...                                                        ..      ...    ...
2022-12-31 Using LLMs to Improve Your Coding               15       15     15
           Using LLMs to Train More LLMs                   15       15     15
           Using LLMs to Win Friends and Influence People  15       15     15
           Using LLMs to Win More Kaggle Competitions      15       15     15
           Using LLMs to Write Better                      15       15     15

[1825 rows x 3 columns]

there are 15 entries per date per product in the test data as well.

In [49]:
train_data.groupby(by=['country','product']).count()

id  date  store  \
country   product                                                             
Argentina Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Canada    Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Estonia   Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Japan     Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   
Spain     Using LLMs to Improve Your Coding               5478  5478   5478   
          Using LLMs to Train More LLMs                   5478  5478   5478   
          Using LLMs to Win Friends and Influence People  5478  5478   5478   
          Using LLMs to Win More Kaggle Competitions      5478  5478   5478   
          Using LLMs to Write Better                      5478  5478   5478   

                                                          num_sold  
country   product                                                   
Argentina Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Canada    Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Estonia   Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Japan     Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478  
Spain     Using LLMs to Improve Your Coding                   5478  
          Using LLMs to Train More LLMs                       5478  
          Using LLMs to Win Friends and Influence People      5478  
          Using LLMs to Win More Kaggle Competitions          5478  
          Using LLMs to Write Better                          5478

there are exactly 5478 entries per country per product in the train data.

In [50]:
test_data.groupby(by=['country','product']).count()

id  date  store
country   product                                                          
Argentina Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Canada    Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Estonia   Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Japan     Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095
Spain     Using LLMs to Improve Your Coding               1095  1095   1095
          Using LLMs to Train More LLMs                   1095  1095   1095
          Using LLMs to Win Friends and Influence People  1095  1095   1095
          Using LLMs to Win More Kaggle Competitions      1095  1095   1095
          Using LLMs to Write Better                      1095  1095   1095

there are exactly 1095 entries per country per product in the train data.

In [51]:
train_data.groupby(by=['country','num_sold']).count()

id  date  store  product
country   num_sold                           
Argentina 2          30    30     30       30
          3         509   509    509      509
          4         693   693    693      693
          5         792   792    792      792
          6         667   667    667      667
...                 ...   ...    ...      ...
Spain     786         1     1      1        1
          793         1     1      1        1
          808         1     1      1        1
          822         1     1      1        1
          827         1     1      1        1

[3263 rows x 4 columns]

there is unequal distribution of sales amongst the country sales in the train data.

In [53]:
train_data.groupby(by=['store','num_sold']).count()

id  date  country  product
store        num_sold                            
Kagglazon    12         1     1        1        1
             13         2     2        2        2
             14        10    10       10       10
             15         7     7        7        7
             16        25    25       25       25
...                    ..   ...      ...      ...
Kaggle Store 354        2     2        2        2
             357        1     1        1        1
             360        1     1        1        1
             362        1     1        1        1
             383        1     1        1        1

[1544 rows x 4 columns]

there is unequal distribution of sales amongst the store sales in the train data.

In [63]:
train_data.groupby(by=['country'])['num_sold'].sum().sort_values(ascending=False)

country
Canada       7013114
Japan        6013786
Spain        4380862
Estonia      3577319
Argentina    1683244
Name: num_sold, dtype: int64

Canada has the highest sales, followed by Japan. Argentina has the least sales. 

In [64]:
train_data.groupby(by=['store'])['num_sold'].sum().sort_values(ascending=False)

store
Kagglazon       15661335
Kaggle Store     4251056
Kaggle Learn     2755934
Name: num_sold, dtype: int64

Kagglazon store has the highest overall sales, followed by Kaggle Store. Kaggle Learn has the least sales. Big difference noticed between Kagglazon and Kaggle Store.

In [65]:
train_data.groupby(by=['product'])['num_sold'].sum().sort_values(ascending=False)

product
Using LLMs to Improve Your Coding                 6069281
Using LLMs to Train More LLMs                     5993975
Using LLMs to Win More Kaggle Competitions        5068313
Using LLMs to Write Better                        4604192
Using LLMs to Win Friends and Influence People     932564
Name: num_sold, dtype: int64

'Using LLMs to Improve Your Coding' product has the highest sales, closely followed by 'Using LLMs to Train More LLMs'.

'Using LLMs to Win Friends and Influence People' is the least sold product. Big difference noticed between the last two products.

In [66]:
train_data.groupby(by=['date'])['num_sold'].sum().sort_values(ascending=False)

date
2018-12-30    21273
2019-12-29    20556
2021-12-31    20422
2021-12-30    20156
2021-12-29    20110
              ...  
2020-04-28     8063
2020-04-21     8000
2020-04-22     7982
2020-04-23     7948
2020-04-27     7942
Name: num_sold, Length: 1826, dtype: int64

Highest sales noticed towards the end of December across years. Least sales noticed towards the end of April 2020.

In [69]:
train_data['date'] = pd.to_datetime(train_data['date'])
train_data.set_index('date', inplace=True)

In [71]:
train_data['num_sold'].resample('Y').sum().sort_values(ascending=False)

date
2021-12-31    4884479
2018-12-31    4723963
2019-12-31    4522172
2017-12-31    4451213
2020-12-31    4086498
Name: num_sold, dtype: int64

the year 2021 witnessed the highest sales. 2020 had the least sales. big drop noticed in the year 2020, followed by a rise in 2021. this might be due to the Covid-19 pandemic.

In [72]:
train_data['num_sold'].resample('M').sum().sort_values(ascending=False)

date
2021-12-31    472927
2021-01-31    437576
2017-12-31    429520
2018-12-31    426613
2018-01-31    414040
2021-03-31    413036
2019-12-31    408632
2021-10-31    408445
2021-11-30    407029
2021-05-31    406520
2019-01-31    405765
2018-05-31    402382
2018-03-31    401631
2017-01-31    401396
2021-07-31    401035
2018-10-31    397466
2021-04-30    397381
2020-12-31    395389
2021-08-31    394822
2018-08-31    393673
2018-07-31    389474
2019-03-31    389053
2021-09-30    388294
2018-04-30    386685
2018-09-30    384665
2019-05-31    384008
2020-01-31    384006
2018-11-30    383739
2019-10-31    383017
2021-06-30    381613
2018-06-30    380247
2017-11-30    379724
2017-10-31    379251
2019-11-30    375945
2021-02-28    375801
2019-08-31    370925
2017-03-31    368364
2019-04-30    365779
2017-05-31    365720
2019-09-30    365085
2017-08-31    364188
2017-07-31    364172
2018-02-28    363348
2020-11-30    363151
2019-07-31    362450
2017-09-30    359522
2019-06-30    358033
2017-04-

most sales seems to have happened in the months of December and January. this could be due to the holiday season Chirstmas and New Year's. Least sales were noticed across months in the year 2020.

In [73]:
train_data['num_sold'].resample('D').sum().sort_values(ascending=False)

date
2018-12-30    21273
2019-12-29    20556
2021-12-31    20422
2021-12-30    20156
2021-12-29    20110
              ...  
2020-04-28     8063
2020-04-21     8000
2020-04-22     7982
2020-04-23     7948
2020-04-27     7942
Name: num_sold, Length: 1826, dtype: int64

most sales noticed in the last days of December (29,30,31) across years. least sales noticed in the month of April in 2020.

there is a hint of seasonality.

In [77]:
train_data.groupby(by=['country','store'])['num_sold'].sum().sort_values(ascending=False)

country    store       
Canada     Kagglazon       4842322
Japan      Kagglazon       4154670
Spain      Kagglazon       3026408
Estonia    Kagglazon       2472309
Canada     Kaggle Store    1316665
Argentina  Kagglazon       1165626
Japan      Kaggle Store    1127598
Canada     Kaggle Learn     854127
Spain      Kaggle Store     821572
Japan      Kaggle Learn     731518
Estonia    Kaggle Store     670621
Spain      Kaggle Learn     532882
Estonia    Kaggle Learn     434389
Argentina  Kaggle Store     314600
           Kaggle Learn     203018
Name: num_sold, dtype: int64

Kagglazon is topping the charts across all countries. Argentina has the least sales in both 'Kaggle Store' and 'Kaggle Learn'

In [75]:
train_data.groupby(by=['country','product'])['num_sold'].sum().sort_values(ascending=False)

country    product                                       
Canada     Using LLMs to Improve Your Coding                 1878940
           Using LLMs to Train More LLMs                     1851838
Japan      Using LLMs to Improve Your Coding                 1608717
           Using LLMs to Train More LLMs                     1592992
Canada     Using LLMs to Win More Kaggle Competitions        1568278
           Using LLMs to Write Better                        1424637
Japan      Using LLMs to Win More Kaggle Competitions        1343553
           Using LLMs to Write Better                        1220446
Spain      Using LLMs to Improve Your Coding                 1173558
           Using LLMs to Train More LLMs                     1158211
           Using LLMs to Win More Kaggle Competitions         979860
Estonia    Using LLMs to Improve Your Coding                  957107
           Using LLMs to Train More LLMs                      946212
Spain      Using LLMs to Write Better        

In Canada and Japan, 'Using LLMs to Improve Your Coding', 'Using LLMs to Train More LLMs' are the best sellers. 

In all countries 'Using LLMs to Improve Your Coding' is one of the best sellers.

In all countries 'Using LLMs to Win Friends and Influence People' is the poorest seller.

In [76]:
train_data.groupby(by=['store','product'])['num_sold'].sum().sort_values(ascending=False)

store         product                                       
Kagglazon     Using LLMs to Improve Your Coding                 4191082
              Using LLMs to Train More LLMs                     4139671
              Using LLMs to Win More Kaggle Competitions        3501840
              Using LLMs to Write Better                        3180699
Kaggle Store  Using LLMs to Improve Your Coding                 1138862
              Using LLMs to Train More LLMs                     1124319
              Using LLMs to Win More Kaggle Competitions         950504
              Using LLMs to Write Better                         863978
Kaggle Learn  Using LLMs to Improve Your Coding                  739337
              Using LLMs to Train More LLMs                      729985
Kagglazon     Using LLMs to Win Friends and Influence People     648043
Kaggle Learn  Using LLMs to Win More Kaggle Competitions         615969
              Using LLMs to Write Better                         559515
Kag

Across all stores 'Using LLMs to Improve Your Coding' is the best seller. 

In [78]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [80]:
result = adfuller(train_data['num_sold'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(f'{key}: {value}')


ADF Statistic: -11.183076035748291
p-value: 2.4731599364612518e-20
Critical Values:
1%: -3.430397776254718
5%: -2.861561116418768
10%: -2.5667812395198735


The results of the Augmented Dickey-Fuller (ADF) test provide information about the stationarity of the sales data:

ADF Statistic: The ADF statistic is a negative value (-11.183076035748291 in our case). The more negative the ADF statistic, the stronger the evidence against the null hypothesis of non-stationarity. In this case, the large negative value suggests that the sales data is likely stationary.

p-value: The p-value associated with the ADF statistic is 2.4731599364612518e-20, which is very close to zero. The p-value represents the probability that the null hypothesis of non-stationarity is true. A low p-value (typically below a significance level of 0.05) suggests rejecting the null hypothesis and concluding that the data is stationary. In this case, the extremely small p-value indicates strong evidence for stationarity.

Critical Values: The critical values at different significance levels (1%, 5%, and 10%) are provided to compare with the ADF statistic. If the ADF statistic is more negative than the critical values, it strengthens the evidence for stationarity. In our case, the ADF statistic is much more negative than all the critical values, further supporting the stationarity of the data.

Based on the ADF statistic, p-value, and critical values, the sales data is likely stationary, indicating that it does not exhibit significant trends or non-stationary patterns.

### Model building

In [81]:
from sklearn.preprocessing import LabelEncoder


In [87]:
label_encoder = LabelEncoder()
labels = ['country', 'store', 'product']

train_data['country_Encoded'] = label_encoder.fit_transform(train_data['country'])
train_data['store_Encoded'] = label_encoder.fit_transform(train_data['store'])
train_data['product_Encoded'] = label_encoder.fit_transform(train_data['product'])

train_data.drop(['id','country', 'store', 'product'], axis=1, inplace=True)

In [94]:
train_data['Year'] = train_data.index.year
train_data['Month'] = train_data.index.month
train_data['Day'] = train_data.index.day

In [95]:
train_data.head()

,num_sold,country_Encoded,store_Encoded,product_Encoded,Year,Month,Day
date,,,,,,,
2017-01-01,63,0,1,0,2017,1,1
2017-01-01,66,0,1,1,2017,1,1
2017-01-01,9,0,1,2,2017,1,1
2017-01-01,59,0,1,3,2017,1,1
2017-01-01,49,0,1,4,2017,1,1


In [96]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [97]:
X = train_data.drop('num_sold', axis=1)
y = train_data['num_sold']

In [98]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and fit the linear regression model
regression_model = LinearRegression()
regression_model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = regression_model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 22518.34321980081
R-squared: 0.3328400189093855


In [114]:
test_data.head()

,Year,Month,Day,country_Encoded,store_Encoded,product_Encoded
date,,,,,,
2022-01-01,2022,1,1,0,1,0
2022-01-01,2022,1,1,0,1,1
2022-01-01,2022,1,1,0,1,2
2022-01-01,2022,1,1,0,1,3
2022-01-01,2022,1,1,0,1,4


In [119]:
test_data = pd.read_csv('/kaggle/input/playground-series-s3e19/test.csv')

In [120]:
test_data['date'] = pd.to_datetime(test_data['date'])
test_data.set_index('date', inplace=True)



test_data['country_Encoded'] = label_encoder.fit_transform(test_data['country'])
test_data['store_Encoded'] = label_encoder.fit_transform(test_data['store'])
test_data['product_Encoded'] = label_encoder.fit_transform(test_data['product'])

test_data['Year'] = test_data.index.year
test_data['Month'] = test_data.index.month
test_data['Day'] = test_data.index.day

test_data.drop(['id','country', 'store', 'product'], axis=1, inplace=True)
test_data_scaled = scaler.transform(test_data)

In [121]:
test_pred = regression_model.predict(test_data_scaled)

In [123]:
test_pred

array([159.67053743, 145.77992804, 131.88931865, ..., 329.15591607,
       315.26530668, 301.37469729])

In [126]:
submission_df= pd.read_csv("/kaggle/input/playground-series-s3e19/sample_submission.csv")
submission_df.columns=["id","num_sold"]
submission_df["num_sold"]=np.expm1(test_pred)
submission_df.to_csv("submission.csv",index=False)